In [1]:
from utils.imports import *

/home/leonard/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Train

In [2]:
src = PATH['model_train_pred']
model_paths = PATH['model_final']
model_fenge_path=model_paths + 'final_fenge_VGG.h5'
model = load_model(model_fenge_path, custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})
#anno = pd.read_csv(csv_path + 'annotations.csv')

## 生成分割结果npy+csv

In [3]:
patients = [x for x in os.listdir(src) if 'orig' in x]

In [4]:
for patient in tqdm(sorted(patients)):
    pred_samples(src,patient,model)

100%|██████████| 800/800 [24:12:13<00:00, 108.92s/it]   


## 获取分割结果csv

In [5]:
patients = [x for x in os.listdir(src) if 'pred.npy' in x]    

In [6]:
empty0 = pandas.DataFrame({'seriesuid':[],'coordX':[],'coordY':[],'coordZ':[],'diameter_mm':[]})
empty1 = pandas.DataFrame({'seriesuid':[],'coordX':[],'coordY':[],'coordZ':[],'diameter_mm':[]})
empty2 = pandas.DataFrame({'seriesuid':[],'coordX':[],'coordY':[],'coordZ':[],'diameter_mm':[]})

for img_file in tqdm(sorted(patients)):
    patient_id = img_file[:-9]
    img_array = np.load(src + img_file)
    #img_array[img_array < 0.5] = 0
    pos_annos = pd.read_csv(src + img_file[:-9] + '_annos_pos.csv')
    origin = np.array([pos_annos.loc[0]['origin_x'],pos_annos.loc[0]['origin_y'],pos_annos.loc[0]['origin_z']]) 
    spacing = np.array([pos_annos.loc[0]['spacing_x'],pos_annos.loc[0]['spacing_y'],pos_annos.loc[0]['spacing_z']])     
    temp = np.squeeze(img_array)
    

    labels0 = skimage.measure.label(np.squeeze(temp))    
    props0 = skimage.measure.regionprops(labels0)
    for i in range(len(props0)):     
        if props0[i]['EquivDiameter'] > 3:
            world_coordinates0 = voxel_2_world([props0[i]['Centroid'][2], 
                                                props0[i]['Centroid'][1], 
                                                props0[i]['Centroid'][0]], origin, spacing)
            insertrow0 = pd.DataFrame([[patient_id,
                                      world_coordinates0[0],
                                      world_coordinates0[1],
                                      world_coordinates0[2],
                                      props0[i]['EquivDiameter']]],columns = ['seriesuid','coordX','coordY','coordZ','diameter_mm'])
        
            empty0 = empty0.append(insertrow0,ignore_index=True)
    
    temp1 = skimage.morphology.opening(np.squeeze(temp), np.ones([3,3,3]))
    labels1 = skimage.measure.label(np.squeeze(temp1))    
    props1 = skimage.measure.regionprops(labels1)
    
    for i in range(len(props1)):     
        if props1[i]['EquivDiameter'] > 3:
            world_coordinates1 = voxel_2_world([props1[i]['Centroid'][2], 
                                                props1[i]['Centroid'][1], 
                                                props1[i]['Centroid'][0]], origin, spacing)               
        
            insertrow1 = pd.DataFrame([[patient_id,
                                      world_coordinates1[0],
                                      world_coordinates1[1],
                                      world_coordinates1[2],
                                      props1[i]['EquivDiameter']]],columns = ['seriesuid','coordX','coordY','coordZ','diameter_mm'])
        
            empty1 = empty1.append(insertrow1,ignore_index=True)
        
    temp2 = skimage.morphology.opening(np.squeeze(temp), np.ones([5,5,5]))
    labels2 = skimage.measure.label(np.squeeze(temp2))    
    props2 = skimage.measure.regionprops(labels2)
    for i in range(len(props2)):
        if props2[i]['EquivDiameter'] > 3:
            world_coordinates2 = voxel_2_world([props2[i]['Centroid'][2], 
                                                props2[i]['Centroid'][1], 
                                                props2[i]['Centroid'][0]], origin, spacing)               
        
            insertrow2 = pd.DataFrame([[patient_id,
                                      world_coordinates2[0],
                                      world_coordinates2[1],
                                      world_coordinates2[2],
                                      props2[i]['EquivDiameter']]],columns = ['seriesuid','coordX','coordY','coordZ','diameter_mm'])
        
            empty2 = empty2.append(insertrow2,ignore_index=True)
            
empty0 = empty0[['seriesuid','coordX','coordY','coordZ','diameter_mm']]
empty1 = empty1[['seriesuid','coordX','coordY','coordZ','diameter_mm']]
empty2 = empty2[['seriesuid','coordX','coordY','coordZ','diameter_mm']] 

100%|██████████| 800/800 [3:10:17<00:00, 14.27s/it]  


In [7]:
empty0.to_csv(src + "0_vgg_final_result.csv", index=False)
empty1.to_csv(src + "1_vgg_final_result.csv", index=False)
empty2.to_csv(src + "2_vgg_final_result.csv", index=False)